In [5]:
import argparse
import torch
from subm_utils.submission_utils import pad

from subm_utils.submission_dataset import get_data_for_evaldataset, EvalDataset, get_dataloaders
# from model.models import get_model
from subm_utils.submission_metrics import produce_submit_file

from model import Model, ModelWithSEMAA
import torchaudio

def get_ssl_aasist_model(ckpt_path, ssl_backbone, ssl_behaviour, device, use_semaa=False):
    '''
    Initializes Rawformer Architecture from the given checkpoint
    '''
    if not use_semaa:
        model = Model(
            args=None,  # not used
            device=device,
            ssl_backbone=ssl_backbone,
            freeze_ssl=False,
            ssl_behaviour=ssl_behaviour
        )
    else:
        model = ModelWithSEMAA(
            args=None,  # not used
            device=device,
            ssl_backbone=ssl_backbone,
            freeze_ssl=False,
            ssl_behaviour=ssl_behaviour
        )
    
    model.load_state_dict(torch.load(ckpt_path, map_location=device))
    
    return model

In [3]:
device = "cpu"

In [4]:
model = get_ssl_aasist_model("models/model_LA_weighted_CCE_100_24_4e-06_wav2vec_NO_AUG_la/ep_3_ckpt.pth", "wav2vec", "last-layer", device)

In [10]:
path = "/data/a.varlamov/LJSpeech-1.1/wavs/LJ001-0011.wav"
audio, sr = torchaudio.load(path)

audio = audio[:, :4 * sr]

In [7]:
batch = torch.cat([audio] * 32)

#### CPU:

In [11]:
%%time
for elem in batch:
    model(elem.unsqueeze(0))

CPU times: user 56min 5s, sys: 4min 18s, total: 1h 23s
Wall time: 35.8 s


In [12]:
%%time
model(batch)

CPU times: user 11min 3s, sys: 4min 4s, total: 15min 8s
Wall time: 7.51 s


tensor([[ 6.6006, -7.3470],
        [ 5.3857, -2.1618],
        [ 3.8108, -1.9464],
        [ 2.4615, -3.3895],
        [ 4.8035, -4.6641],
        [ 4.6274, -3.8218],
        [ 4.8398, -4.4488],
        [ 3.4979, -3.9882],
        [ 4.8715, -5.3953],
        [ 4.3552, -5.3121],
        [ 3.6739, -4.2780],
        [ 0.8629, -1.1323],
        [ 4.1811, -3.1909],
        [ 5.2168, -2.2710],
        [ 4.7340, -3.3822],
        [ 2.3148, -6.8783],
        [ 1.8244, -2.7058],
        [ 4.0966, -1.0702],
        [ 2.8876, -3.2097],
        [ 3.2593, -1.8153],
        [ 9.3529, -3.7881],
        [ 6.1222, -0.8167],
        [ 5.0184, -6.0105],
        [ 4.6984, -4.4686],
        [ 2.3077, -1.9260],
        [ 2.7475, -2.8326],
        [ 8.0732, -7.7983],
        [ 5.0947, -2.4320],
        [ 4.6018, -4.0262],
        [ 1.9857, -2.9927],
        [ 2.7098, -2.1089],
        [ 2.4642, -3.1123]], grad_fn=<AddmmBackward>)

#### GPU:

In [15]:
device = "cuda:5"

In [16]:
model = model.to(device)
batch = batch.to(device)

In [17]:
%%time
for elem in batch:
    model(elem.unsqueeze(0))

CPU times: user 1.32 s, sys: 95.7 ms, total: 1.42 s
Wall time: 1.48 s


In [24]:
%%time
model(batch)

RuntimeError: CUDA out of memory. Tried to allocate 124.00 MiB (GPU 5; 79.15 GiB total capacity; 67.87 GiB already allocated; 112.12 MiB free; 68.05 GiB reserved in total by PyTorch)